In [15]:
import pandas as pd
import numpy as np
import sqlite3
import re


In [16]:
sql_file = open(r"database\ddl\create_tables.sql",encoding="ISO-8859-1")
sql_as_string = sql_file.read()
sql_file.close()

In [17]:
ddl_tables = dict()
relation_tables = dict()

regex_create_table = r"\bCREATE\s+TABLE\s+\[\w+\]\.\[\w+\]\s*\(\s*([^;]+)\s*\);"
regex_name_table = r"\bCREATE\s+TABLE\s+\[(\w+)\]\.\[(\w+)\]"
regex_relations = r'FOREIGN KEY\((\w+)\)\s+REFERENCES\s+(\w+)\((\w+)\)'

matches = re.finditer(regex_create_table, sql_as_string)

regex_name_table = r"\bCREATE\s+TABLE\s+\[(\w+)\]\.\[(\w+)\]"

for matche in matches:
	list_relations = list()

	name_table = re.search(regex_name_table, matche.group())
	relations = re.finditer(regex_relations, matche.group())
	for relation in relations:
		list_relations.append([{name_table.group(2):relation.group(1)},{relation.group(2):relation.group(3)}])

     
	ddl_tables[name_table.group(2)]  = matche.group()
	relation_tables[name_table.group(2)] = list_relations


In [18]:
file = r"C:\Users\roger\Documents\trabalhofinalPECE\Metadados_preenchido.xlsx"

In [19]:
df = pd.read_excel(file,sheet_name="Resumo das tabelas")

In [20]:
def get_info_column(file,sheet):
    output = dict()
    df_table = pd.read_excel(file,sheet_name=sheet[:31])
    for i,row in df_table.iterrows():
        output.update({row['Campo']:{"Descrição":row['Descrição']}})

    return output


In [21]:

conn = sqlite3.connect("database\db\dbo.db")
metadata_json = dict()
for i,row in df.iterrows():
    metadata_json.update({row["Nome"]:{nm:vl if vl else "" for nm,vl in row.items() if nm != "Nome"} })
    metadata_json[row["Nome"]].update({"Colunas":get_info_column(file,row["Nome"])})
    metadata_json[row["Nome"]].update({"DDL":ddl_tables.get(row["Nome"],'')})
    metadata_json[row["Nome"]].update({"Relações":relation_tables.get(row["Nome"],'')})

    sql_query = f"""SELECT * FROM {row['Nome']};"""
    df_by_sql = pd.read_sql(sql_query,conn)
    desc = df_by_sql.describe(include='all')
    only_unique = desc.loc[desc.index.isin(["unique"])].T.reset_index()
    lista_unique = only_unique.loc[only_unique['unique']<=5,'index'].values.tolist()
    if len(lista_unique)>1:
        for coluna in lista_unique:
            unique_by_col_sql = pd.read_sql(f"""SELECT DISTINCT {coluna} FROM {row["Nome"]}""",conn)
            metadata_json[row["Nome"]]['Colunas'][coluna].update({"Valores Possíveis":unique_by_col_sql[coluna].values.tolist()}) 

    

In [22]:

import json 
# Convert and write JSON object to file
with open("dicionario_dados.json", "w") as outfile: 
    json.dump(metadata_json, outfile)

In [23]:
file = r"C:\Users\roger\Documents\trabalhofinalPECE\querys.xlsx"
df = pd.read_excel(file,sheet_name='Planilha1')
df_to_use = df.loc[:,["Question","NEW INTENT","TABLE","COMPLEXIDADE"]].dropna()
data=df_to_use.rename(columns = {'NEW INTENT': 'label', 'Question': 'text','TABLE':'tables_related'}, inplace = False)

In [24]:
data[["label","tables_related","COMPLEXIDADE"]].drop_duplicates().to_dict("records")

[{'label': 'CARGO_DO_FUNCIONARIO',
  'tables_related': 'DimEmployee',
  'COMPLEXIDADE': 1},
 {'label': 'DEPARTAMENTO_DO_FUNCIONARIO',
  'tables_related': 'DimEmployee',
  'COMPLEXIDADE': 1},
 {'label': 'PRIMEIRA_DTCOMPRA_CLIENTE',
  'tables_related': 'DimCustomer',
  'COMPLEXIDADE': 1},
 {'label': 'ULTIMA_DTCOMPRA_CLIENTE',
  'tables_related': 'FactInternetSales, DimDate, DimCustomer',
  'COMPLEXIDADE': 3},
 {'label': 'INFORMACAO_PESSOAL_CLIENTE',
  'tables_related': 'DimCustomer',
  'COMPLEXIDADE': 1},
 {'label': 'LOCALIZAR_CLIENTE_POR_INFO_PESSOAIS',
  'tables_related': 'DimCustomer',
  'COMPLEXIDADE': 2},
 {'label': 'LOCALIZAR_FUNCIONARIO',
  'tables_related': 'DimEmployee',
  'COMPLEXIDADE': 2},
 {'label': 'LOCALIZAR_PRODUTOS_PELA_PRODUCAO',
  'tables_related': 'DimProduct',
  'COMPLEXIDADE': 2},
 {'label': 'LOCALIZAR_PRODUTOS_PELO_CUSTO',
  'tables_related': 'DimProduct',
  'COMPLEXIDADE': 2},
 {'label': 'METRICAS_DE_CUSTO_DE_PRODUTOS',
  'tables_related': 'DimProduct',
  'COMPLEX

In [25]:
dict_output = dict()
for i,row in data[["label","tables_related"]].drop_duplicates().iterrows():
    dict_output.update({row['label']:[table.replace(' ','') for table in row['tables_related'].split(",")]})


In [26]:
dict_output

{'CARGO_DO_FUNCIONARIO': ['DimEmployee'],
 'DEPARTAMENTO_DO_FUNCIONARIO': ['DimEmployee'],
 'PRIMEIRA_DTCOMPRA_CLIENTE': ['DimCustomer'],
 'ULTIMA_DTCOMPRA_CLIENTE': ['FactInternetSales', 'DimDate', 'DimCustomer'],
 'INFORMACAO_PESSOAL_CLIENTE': ['DimCustomer'],
 'LOCALIZAR_CLIENTE_POR_INFO_PESSOAIS': ['DimCustomer'],
 'LOCALIZAR_FUNCIONARIO': ['DimEmployee'],
 'LOCALIZAR_PRODUTOS_PELA_PRODUCAO': ['DimProduct'],
 'LOCALIZAR_PRODUTOS_PELO_CUSTO': ['DimProduct'],
 'METRICAS_DE_CUSTO_DE_PRODUTOS': ['DimProduct'],
 'METRICAS_DE_PRODUCAO_DE_PRODUTOS': ['DimProduct'],
 'METRICAS_DE_PRODUTOS_POR_CATEGORIA': ['DimProduct',
  'DimProductCategory',
  'DimProductSubcategory'],
 'METRICAS_DO_QUADRO_DE_FUNCIONARIOS': ['DimEmployee'],
 'METRICAS_DOS_CLIENTES': ['DimCustomer'],
 'MOVIMENTACAO_QUADRO_DE_FUNCIONARIOS': ['DimEmployee'],
 'QUANTIDADE_DE_DEPARTAMENTOS': ['DimEmployee'],
 'TEMPO_DE_CASA_FUNCIONARIO': ['DimEmployee']}

In [27]:
import json 
# Convert and write JSON object to file
with open("AdventureQI.json", "w") as outfile: 
    json.dump(dict_output, outfile)

In [28]:
dict_output

{'CARGO_DO_FUNCIONARIO': ['DimEmployee'],
 'DEPARTAMENTO_DO_FUNCIONARIO': ['DimEmployee'],
 'PRIMEIRA_DTCOMPRA_CLIENTE': ['DimCustomer'],
 'ULTIMA_DTCOMPRA_CLIENTE': ['FactInternetSales', 'DimDate', 'DimCustomer'],
 'INFORMACAO_PESSOAL_CLIENTE': ['DimCustomer'],
 'LOCALIZAR_CLIENTE_POR_INFO_PESSOAIS': ['DimCustomer'],
 'LOCALIZAR_FUNCIONARIO': ['DimEmployee'],
 'LOCALIZAR_PRODUTOS_PELA_PRODUCAO': ['DimProduct'],
 'LOCALIZAR_PRODUTOS_PELO_CUSTO': ['DimProduct'],
 'METRICAS_DE_CUSTO_DE_PRODUTOS': ['DimProduct'],
 'METRICAS_DE_PRODUCAO_DE_PRODUTOS': ['DimProduct'],
 'METRICAS_DE_PRODUTOS_POR_CATEGORIA': ['DimProduct',
  'DimProductCategory',
  'DimProductSubcategory'],
 'METRICAS_DO_QUADRO_DE_FUNCIONARIOS': ['DimEmployee'],
 'METRICAS_DOS_CLIENTES': ['DimCustomer'],
 'MOVIMENTACAO_QUADRO_DE_FUNCIONARIOS': ['DimEmployee'],
 'QUANTIDADE_DE_DEPARTAMENTOS': ['DimEmployee'],
 'TEMPO_DE_CASA_FUNCIONARIO': ['DimEmployee']}